In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Esophageal_Cancer"
cohort = "GSE218109"

# Input paths
in_trait_dir = "../../input/GEO/Esophageal_Cancer"
in_cohort_dir = "../../input/GEO/Esophageal_Cancer/GSE218109"

# Output paths
out_data_file = "../../output/preprocess/Esophageal_Cancer/GSE218109.csv"
out_gene_data_file = "../../output/preprocess/Esophageal_Cancer/gene_data/GSE218109.csv"
out_clinical_data_file = "../../output/preprocess/Esophageal_Cancer/clinical_data/GSE218109.csv"
json_path = "../../output/preprocess/Esophageal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Esophageal Squamous Cell Carcinoma tumors from Indian patients: nuclear-stabilized p53 (NS+) versus unstable p53 (NS-) protein"
!Series_summary	"Transcriptional profiling of Esophageal Squamous Cell Carcinoma (ESCC) tumors comparing samples harbouring nuclear-stabilized p53 (NS+) versus unstable p53 (NS-) protein, determined through immunohistochemistry (IHC) staining of the tumor sections. The goal was to identify the genes that were differentially regulated between NS+ and NS- ESCC samples."
!Series_overall_design	"Two-condition experiment, NS+ versus NS- esophageal tumors. NS+ tumors: 17, NS- tumors: 19."
Sample Characteristics Dictionary:
{0: ['Sex: M', 'Sex: F'], 1: ['age: 22', 'age: 45', 'age: 52', 'age: 50', 'age: 34', 'age: 55', 'age: 48', 'age: 64', 'age: 70', 'age: 68', 'age: 23', 'age: 62', 'age: 59', 'age: 58', 'age: 41', 'age: 47', 'age: 66', 'age: 38', 'age: 79', 'age: 61', 'age: 39', 'age: 32', 'age: 46', 'age: 69', 'age: 54'], 2: [

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# comparing esophageal tumor samples with different p53 status
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: p53 status in row 5
trait_row = 5
# For age: age in row 1
age_row = 1
# For gender: sex in row 0
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(val):
    """Convert p53 status to binary (0 for NS-, 1 for NS+)"""
    if not isinstance(val, str):
        return None
    
    val = val.lower()
    if "p53 status:" in val:
        val = val.split("p53 status:")[1].strip()
    
    if "nuclear-stabilized" in val or "ns+" in val:
        return 1
    elif "unstable" in val or "ns-" in val:
        return 0
    return None

def convert_age(val):
    """Convert age to numeric value"""
    if not isinstance(val, str):
        return None
    
    if "age:" in val:
        try:
            age = int(val.split("age:")[1].strip())
            return age
        except:
            pass
    return None

def convert_gender(val):
    """Convert gender to binary (0 for female, 1 for male)"""
    if not isinstance(val, str):
        return None
    
    val = val.lower()
    if "sex:" in val:
        val = val.split("sex:")[1].strip()
    
    if val == 'f' or val == 'female':
        return 0
    elif val == 'm' or val == 'male':
        return 1
    return None

# 3. Save Metadata
# Trait data availability is determined by whether trait_row is None
is_trait_available = trait_row is not None

# Use the validate_and_save_cohort_info function for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

def get_feature_data(clinical_df, row_idx, feature_name, convert_func):
    """Helper function to extract and process feature data"""
    feature_values = clinical_df.iloc[row_idx].tolist()
    converted_values = [convert_func(val) for val in feature_values]
    return pd.DataFrame({feature_name: converted_values})

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a sample DataFrame based on the characteristics data where:
    # - Each column is a sample
    # - Each row represents a different characteristic (indexed by the keys in sample_chars_dict)
    sample_chars_dict = {0: ['Sex: M', 'Sex: F'], 
                         1: ['age: 22', 'age: 45', 'age: 52', 'age: 50', 'age: 34', 'age: 55', 'age: 48', 
                             'age: 64', 'age: 70', 'age: 68', 'age: 23', 'age: 62', 'age: 59', 'age: 58', 
                             'age: 41', 'age: 47', 'age: 66', 'age: 38', 'age: 79', 'age: 61', 'age: 39', 
                             'age: 32', 'age: 46', 'age: 69', 'age: 54'], 
                         2: ['tissue: Esophageal Squamous Cell Carcinoma'], 
                         3: ['Stage: pT3N2', 'Stage: pT3N0', 'Stage: pT3N1', 'Stage: pT3N1bM1b', 'Stage: pT2PN1a', 
                             'Stage: pT2N0Mx', 'Stage: pT2N2', 'Stage: NA', 'Stage: pT2N0', 'Stage: pT2N1b', 
                             'Stage: pT3N1Mx', 'Stage: pT3N2Mx', 'Stage: pT2N1', 'Stage: pT3N0Mx'], 
                         4: ['grade: I', 'grade: II'], 
                         5: ['p53 status: unstable p53 (NS-)', 'p53 status: nuclear-stabilized p53 (NS+)']}
    
    # Extract individual features directly
    feature_list = []
    
    # Extract trait data
    trait_values = sample_chars_dict[trait_row]
    trait_converted = [convert_trait(val) for val in trait_values]
    trait_df = pd.DataFrame({trait: trait_converted})
    feature_list.append(trait_df)
    
    # Extract age data if available
    if age_row is not None:
        age_values = sample_chars_dict[age_row]
        age_converted = [convert_age(val) for val in age_values]
        age_df = pd.DataFrame({'Age': age_converted})
        feature_list.append(age_df)
    
    # Extract gender data if available
    if gender_row is not None:
        gender_values = sample_chars_dict[gender_row]
        gender_converted = [convert_gender(val) for val in gender_values]
        gender_df = pd.DataFrame({'Gender': gender_converted})
        feature_list.append(gender_df)
    
    # Combine all features
    # Note: This will align data by index, effectively creating a proper clinical DataFrame
    selected_clinical_df = pd.concat(feature_list, axis=1)
    
    # Preview the selected clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical data:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the selected clinical data to a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{'Esophageal_Cancer': [0.0, 1.0, nan, nan, nan], 'Age': [22, 45, 52, 50, 34], 'Gender': [1.0, 0.0, nan, nan, nan]}
Clinical data saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE218109.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 67
Header line: "ID_REF"	"GSM6734720"	"GSM6734721"	"GSM6734722"	"GSM6734723"	"GSM6734724"	"GSM6734725"	"GSM6734726"	"GSM6734727"	"GSM6734728"	"GSM6734729"	"GSM6734730"	"GSM6734731"	"GSM6734732"	"GSM6734733"	"GSM6734734"	"GSM6734735"	"GSM6734736"	"GSM6734737"	"GSM6734738"	"GSM6734739"	"GSM6734740"	"GSM6734741"	"GSM6734742"	"GSM6734743"	"GSM6734744"	"GSM6734745"	"GSM6734746"	"GSM6734747"	"GSM6734748"	"GSM6734749"	"GSM6734750"	"GSM6734751"	"GSM6734752"	"GSM6734753"	"GSM6734754"	"GSM6734755"
First data line: 12	9.15E+02	1.50E+03	2.05E+03	1.77E+03	1.19E+03	2.75E+03	6.58E+02	1.53E+03	7.63E+02	2.48E+03	1.23E+03	1.33E+03	1.11E+03	1.14E+04	2.24E+03	5.97E+03	2.53E+03	1.43E+03	4.77E+02	3.44E+03	5.13E+03	2.82E+03	4.34E+03	9.98E+02	1.09E+03	6.81E+03	9.47E+02	2.08E+03	1.45E+03	4.91E+03	2.11E+03	1.40E+01	1.14E+03	3.27E+03	2.21E+03	3.34E+03
Index(['12', '14', '15', '16', '17', '18', '19', '20', '22', '23', '24', '25',
       '26', '27', '30', '33', '35', '36', '37', '38'],
  

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the expression data
# The identifiers appear to be numeric values (12, 14, 15, 16...) which are not standard human gene symbols
# These are likely to be probe IDs or some other platform-specific identifiers 
# that need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE218109
Line 6: !Series_title = Esophageal Squamous Cell Carcinoma tumors from Indian patients: nuclear-stabilized p53 (NS+) versus unstable p53 (NS-) protein
Line 7: !Series_geo_accession = GSE218109
Line 8: !Series_status = Public on Mar 29 2024
Line 9: !Series_submission_date = Nov 16 2022
Line 10: !Series_last_update_date = Mar 30 2024
Line 11: !Series_pubmed_id = 38358025
Line 12: !Series_summary = Transcriptional profiling of Esophageal Squamous Cell Carcinoma (ESCC) tumors comparing samples harbouring nuclear-stabilized p53 (NS+) versus unstable p53 (NS-) protein, determined through immunohistochemistry (IHC) staining of the tumor sections. The goal was to identify the genes that were differentiall


Gene annotation preview:
{'ID': [1, 2, 3, 4, 5], 'COL': [266, 266, 266, 266, 266], 'ROW': [170, 168, 166, 164, 162], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan, nan, nan], 'ORDER': [1, 2, 3, 4, 5]}


### Step 6: Gene Identifier Mapping

In [7]:
# Identify which columns in gene_annotation contain the probe IDs and gene symbols
# From the preview, 'ID' column appears to contain numeric identifiers matching those in gene_data
# 'GENE_SYMBOL' appears to be the column containing gene symbols

# 1. Get the mapping between probe IDs and gene symbols
probe_col = "ID"
gene_col = "GENE_SYMBOL"

# First, let's check if these columns exist in our annotation data
print(f"Columns in gene_annotation: {gene_annotation.columns.tolist()}")

# 2. Get the gene mapping dataframe by extracting the two relevant columns
mapping_data = gene_annotation[[probe_col, gene_col]].copy()
mapping_data = mapping_data.dropna(subset=[gene_col])  # Drop rows with no gene symbol
mapping_data = mapping_data.astype({probe_col: 'str'})  # Convert ID to string to match gene_data index

# Rename the gene symbol column to 'Gene' to match the expected column name in apply_gene_mapping
mapping_data = mapping_data.rename(columns={gene_col: 'Gene'})

# Preview the mapping
print("\nGene mapping preview (first 5 rows):")
print(mapping_data.head())

# Check how many probes have gene symbols
print(f"\nTotal probes in annotation: {len(gene_annotation)}")
print(f"Probes with gene symbols: {len(mapping_data)}")

# 3. Apply the gene mapping to convert probe-level data to gene-level data
# Using the library function to handle the many-to-many mapping
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Check the resulting gene expression data
print("\nGene expression data preview (first 5 genes):")
print(gene_data.head())
print(f"Total number of genes in expression data: {len(gene_data)}")

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Columns in gene_annotation: ['ID', 'COL', 'ROW', 'NAME', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'GENE', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'TIGR_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE', 'SPOT_ID.1', 'ORDER']

Gene mapping preview (first 5 rows):
    ID          Gene
11  12      APOBEC3B
13  14        ATP11B
14  15  LOC100132006
15  16        DNAJA1
17  18         EHMT2

Total probes in annotation: 45220
Probes with gene symbols: 32696

Gene expression data preview (first 5 genes):
       GSM6734720  GSM6734721  GSM6734722  GSM6734723  GSM6734724  GSM6734725  \
Gene                                                                            
A1BG       5040.0      1800.0      3190.0      3580.0       873.0      5500.0   
A1CF         31.0       137.0        25.3        30.3        42.9        12.7   
A2LD1      1250.0       785.0       791.0       625.0       829.0       446.0   
A2M      126000.0    238


Gene expression data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE218109.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (14998, 36)
First few genes with their expression values after normalization:
        GSM6734720  GSM6734721  GSM6734722  GSM6734723  GSM6734724  \
Gene                                                                 
A1BG        5040.0      1800.0      3190.0      3580.0       873.0   
A1CF          31.0       137.0        25.3        30.3        42.9   
A2M       126000.0    238000.0     93400.0     19100.0      8470.0   
A2ML1       1390.0      3640.0       611.0      1050.0      1790.0   
A4GALT       576.0      1140.0       732.0       924.0        76.3   

        GSM6734725  GSM6734726  GSM6734727  GSM6734728  GSM6734729  ...  \
Gene                                                                ...   
A1BG        5500.0      3420.0      1020.0       667.0      2390.0  ...   
A1CF          12.7        15.7        69.9        61.4        32.8  ...   
A2M       107000.0    122000.0     56500.0     46400.0     23200.0  ...   
A2ML1        148.0       267

Normalized gene data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE218109.csv
Raw clinical data shape: (6, 37)
Clinical features:
                   GSM6734720  GSM6734721  GSM6734722  GSM6734723  GSM6734724  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         0.0   
Age                      22.0        45.0        52.0        50.0        34.0   
Gender                    1.0         1.0         0.0         0.0         0.0   

                   GSM6734725  GSM6734726  GSM6734727  GSM6734728  GSM6734729  \
Esophageal_Cancer         0.0         0.0         0.0         0.0         1.0   
Age                      55.0        48.0        64.0        70.0        68.0   
Gender                    1.0         0.0         1.0         1.0         0.0   

                   ...  GSM6734746  GSM6734747  GSM6734748  GSM6734749  \
Esophageal_Cancer  ...         1.0         1.0         0.0         0.0   
Age                ...        59.0        39.0        

Data shape after handling missing values: (36, 15001)
For the feature 'Esophageal_Cancer', the least common label is '1.0' with 17 occurrences. This represents 47.22% of the dataset.
The distribution of the feature 'Esophageal_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 44.0
  50% (Median): 53.0
  75%: 62.0
Min: 22.0
Max: 79.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 14 occurrences. This represents 38.89% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Esophageal_Cancer/GSE218109.csv
